In [10]:
import chess
import chess.uci
import chess.pgn
import os
import csv
import matplotlib.pyplot as plt
from numpy import array
import numpy as np
import math
import time
from multiprocessing import Pool


In [11]:
#fileDir = os.path.dirname(os.path.realpath('__file__'))
pgns_all_path = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/lichess_db_standard_rated_2017-03.pgn"
#abs_file_path = os.path.join(fileDir, rel_path)
#pngs = open(abs_file_path, 'r')

pgns = open(pgns_all_path)


writing_path = "/Users/tylerahlstrom/Documents/GitHub/DI_Proposal/data/stockfish_performances_fifteenseconds.txt"
stockfish_performances = open(writing_path, 'a')


In [13]:
handler = chess.uci.InfoHandler()
engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
engine.info_handlers.append(handler)

In [14]:
evaltime = 100 #ms
#evaluation = engine.go(movetime=evaltime)

In [15]:
def check_elos(game):
    print("white elo: " + game.headers["WhiteElo"])
    print("black elo: " + game.headers["BlackElo"])

In [16]:
def get_performance_dicts(game): 
    handler = chess.uci.InfoHandler()
    engine = chess.uci.popen_engine("/Applications/Stockfish/src/stockfish")
    engine.info_handlers.append(handler)
    
    board = game.board()

    score = 0
    prev_score = 0
    wp = {}
    bp = {}
    evals = []
    
    
    #wp.append(game.headers["WhiteElo"])
    #bp.append(game.headers["BlackElo"])
    wp["elo"] = game.headers["WhiteElo"]
    wp["opp_elo"] = game.headers["BlackElo"]

    bp["elo"] = game.headers["BlackElo"]
    bp["opp_elo"] = game.headers["WhiteElo"]
    
    
    result = game.headers["Result"]
    r1, r2 = interpret_result(result)
    
#     wp.append(r1)
#     bp.append(r2)
    wp["result"] = r1
    bp["result"] = r2
    
#     wp.append(game.headers["ECO"])
#     bp.append(game.headers["ECO"])

    wp["opening"] = game.headers["ECO"]
    bp["opening"] = game.headers["ECO"]

    evaluation = engine.go(movetime=evaltime)
    time.sleep(evaltime/1000)
    score = handler.info["score"][1].cp
    evals.append(score)

    move_count = 0
    prev_score = score
    
#     print("Beginning game with score: " + str(prev_score))
    
    wp_moves = []
    bp_moves = []
    
    for move in game.main_line():
        board.push(move)
        engine.position(board)
        move_count += 1
        evaluation = engine.go(movetime=evaltime)
        time.sleep(evaltime/1000)

        score = handler.info["score"][1].cp
        if score is None:
#             print("Finishing game...")
            break
        if move_count % 2 == 1:
            score = -score
        evals.append(score)
        dif = score - prev_score
        # print("dif calculated as: " + str(score) + " minus " + str(prev_score))
        # performance.append("%.2f" % dif)
        if move_count % 2 == 1:
            wp_moves.append("%.2f" % dif)
        elif move_count % 2 == 0:
            bp_moves.append("%.2f" % -dif)
        prev_score = score

    wp["moves"] = wp_moves
    wp["opp_moves"] = bp_moves
    
    bp["moves"] = bp_moves
    bp["opp_moves"] = wp_moves

#     print(wp)
#     print(bp)
    return [wp, bp]

In [17]:
def interpret_result(result):
    r1 = -1.0
    r2 = -1.0
    if len(result) == 3:
        r1 = float(result[0])
        r2 = float(result[2])
    if len(result) == 7:
        r1 = 0.5
        r2 = 0.5
    assert r1 != -1.0, "Bad result detected"
    assert r2 != -1.0, "Bad result detected"

    return r1, r2

In [18]:
def skip_to_game(pgn_file, game_num):

    offsets = chess.pgn.scan_offsets(pgn_file)
    for i in range(game_num):
        book_mark = next(offsets)
        if i == (game_num - 1):
            pgn_file.seek(book_mark)
    #return pgn_file
# first_game_offset = next(offsets)
# second_game_offset = next(offsets)






In [ ]:
pgns = open(pgns_all_path)
skip_to_game(pgns, 0)



for i in range(1):
    game = chess.pgn.read_game(pgns)
    check_elos(game)
    

    

In [19]:
def evaluate_game(game):
#     starting_position = 0
#     with open(pgns_all_path) as pgns:
#         skip_to_game(pgns, starting_position + i)
    
#     for i in range(amount):

#         #pgns = open(pgns_all_path)

#         game = chess.pgn.read_game(pgns)
        performances = get_performance_dicts(game)

        for p in performances:
            stockfish_performances.write(str(p) + "\n")
        stockfish_performances.flush()

        print("Finished a game")


In [ ]:
batch_size = 500

if __name__ == '__main__':
    
    raw_games = []
    for i in range(batch_size):
        pgn = chess.pgn.read_game(pgns)
        raw_games.append(pgn)
    start_time = time.time()
    pool = Pool(8)
    for evaluated_game in pool.map(evaluate_game,raw_games):
        print(time.time() - start_time)

Finished a game
Finished a game
Finished a game
Finished a game
Finished a game


Process PoolWorker-6:
Process PoolWorker-7:
Process PoolWorker-5:
Process PoolWorker-2:
Process PoolWorker-3:
Process PoolWorker-1:
Process PoolWorker-4:
Process PoolWorker-8:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multi

  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
    evaluation = engine.go(movetime=evaltime)
    return self._queue_command(command, async_callback)
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    return self._queue_command(command, async_callback)
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/sit

Finished a game
Finished a game


Process PoolWorker-15:
Process PoolWorker-16:
Process PoolWorker-11:
Process PoolWorker-9:
Process PoolWorker-10:
Process PoolWorker-12:
Traceback (most recent call last):
Process PoolWorker-14:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-13:
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line

...

  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
    evaluation = engine.go(movetime=evaltime)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
    return self._queue_command(command, async_callback)
    return self._queue_command(command, async_callback)
    return future.result(timeout=FUTURE_POLL_TIMEOUT)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    return self._queue_command(command, async_callback)
    return self.

Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game
Finished a game


Process PoolWorker-29:
Process PoolWorker-26:
Process PoolWorker-32:
Process PoolWorker-28:
Process PoolWorker-25:
Process PoolWorker-31:
Process PoolWorker-27:
Process PoolWorker-30:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/

    evaluation = engine.go(movetime=evaltime)
  File "<ipython-input-16-4766d7e2fea9>", line 55, in get_performance_dicts
    return self._queue_command(command, async_callback)
    evaluation = engine.go(movetime=evaltime)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    evaluation = engine.go(movetime=evaltime)
    return future.result(timeout=FUTURE_POLL_TIMEOUT)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 666, in _queue_command
    evaluation = engine.go(movetime=evaltime)
    return self._queue_command(command, async_callback)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
    return future.result(timeout=FUTURE_POLL_TIMEOUT)
  File "/usr/local/lib/python2.7/site-packages/chess/uci.py", line 997, in go
  File "/usr/loc

    self._target(*self._args, **self._kwargs)
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py", line 376, in get
    task = get()
    task = get()
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "/usr/local/Cellar/python/2.7.11/Frameworks/Python.framework/Versions/2.7/lib/python2.7/multiprocessing/queues.py", line 376, in get
    racquire()
  File "/us